In [ ]:
import pandas as pd
import numpy as np
import os
cwd = os.getcwd()
cwd

In [ ]:
PATH = '/data22/bde'
download = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Download/Driving/'
df_list = []
file_dir = download

for file in os.listdir(file_dir)[:]:
    print(file)
    df = pd.read_csv(file_dir+file)
    #print(df.head())
    
    df = df[['Timestamp', 'Speed', 'CellID', 'RSRP', 'RSRQ', 'SNR', 'CQI', 'RSSI', 'DL_bitrate', 'UL_bitrate']]
    df = df.replace('-', np.nan)
    
    df['date'] = pd.to_datetime(df['Timestamp'], format="%Y.%m.%d_%H.%M.%S")
    df = df.drop(columns=['Timestamp'])

    df.fillna(method='ffill', inplace=True)
    #df.fillna(method='ffill', inplace=True, limit=5)
    df.fillna(0, inplace=True)

    df['CellID'] = pd.to_numeric(df['CellID'])
    
    #df['RSRP'] = pd.to_numeric(df['RSRP'], downcast='integer')
    df['RSRP'] = pd.to_numeric(df['RSRP'])
    
    df['RSRQ'] = pd.to_numeric(df['RSRQ'])
    df['SNR'] = pd.to_numeric(df['SNR'], downcast='integer')
    df['CQI'] = pd.to_numeric(df['CQI'], downcast='integer')
    df['RSSI'] = pd.to_numeric(df['RSSI'], downcast='integer')
    df['DL_bitrate'] = pd.to_numeric(df['DL_bitrate'])
    df['UL_bitrate'] = pd.to_numeric(df['UL_bitrate'])

    df.drop_duplicates(subset=['date'], inplace=True)
    df.info()
    #df.info(show_counts=True)
    #print(df.head())
    #df.plot(subplots=True, layout=(3,3), figsize=(40, 25))
    #df.to_csv('5Gdataset-{}.csv'.format(file), encoding='utf-8', index=False)
    df_list.append(df)

In [ ]:
df = pd.concat(df_list, axis=0)
df.sort_values('date', inplace=True)
df.info()

In [ ]:
df.head()

In [ ]:
df = df[['date', 'Speed', 'CellID', 'RSRP', 'RSRQ', 'SNR', 'CQI', 'RSSI', 'DL_bitrate', 'UL_bitrate']]
df.info()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def multi_corr(feature):
    fig, ax = plt.subplots(1,3, figsize=(20, 6))

    # when the Pearson and Spearman values are not much different, 
    # our data tends to not have extreme values (outliers)
    corr1 = df.corr('pearson')[[feature]].sort_values(by=feature, ascending=False)
    corr2 = df.corr('spearman')[[feature]].sort_values(by=feature, ascending=False)

    corr3 = df.corr('kendall')[[feature]].sort_values(by=feature, ascending=False)
    #ordinal correlation (Spearman & Kendall Tau)

    sns.heatmap(corr1, ax=ax[0], annot=True)
    sns.heatmap(corr2, ax=ax[1], annot=True)
    sns.heatmap(corr3, ax=ax[2], annot=True)

In [ ]:
multi_corr('DL_bitrate')

In [ ]:
df.to_csv('data/5Gdataset/download.csv', encoding='utf-8', index=False)

In [ ]:
# number of changes in values
(df != df.shift(axis=0)).sum(axis=0)